# get predictions from final models

In [1]:
import gc
import h5py
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import Input, Dense, Dropout
from keras import optimizers, regularizers, losses

from keras.models import Model
from keras import backend as K
from keras.callbacks import CSVLogger
from keras import metrics

import scipy
import datetime 

import h5py
import keras

import sys

from sklearn.model_selection import ParameterGrid
import pickle


import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

K.tensorflow_backend._get_available_gpus()

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

sys.path.append("/home/nbbwang/bigdrive/AD_Project/")
from IntegratedGradients import *

/home/nbbwang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from matplotlib import pyplot as plt
%matplotlib inline


import keras 

from scipy import stats 

path_to_configs = "../"
sys.path.append(path_to_configs)
from configs import *
from models import *

In [3]:
dset = 'Blood_GSE63060'
dpath = "_intersection"


path_to_data = path_to_configs + path_to_ext_val_data_folder + "processed_intersection/%s/"%dset
X_test = np.loadtxt(path_to_data + "GE_PCA_test.txt")
X_train = np.loadtxt(path_to_data + "GE_PCA_train.txt")
    
    
path_to_final_models_MTL = "intersecting_gene_models/"

In [4]:
MTL_final_final_model = pickle.load(open(path_to_configs+path_to_final_models_chosen + "MTL/final.p", "rb" ) )

baselines_final_final_model = pickle.load(open(path_to_configs+path_to_final_models_chosen + "MLP_baselines/final.p", "rb" ) )

In [5]:
MTL_phenotype_output_mapping = {"BRAAK":0, "CERAD":1, "PLAQUES":2, "TANGLES":3, "ABETA_IHC":4, "TAU_IHC":5}
phens = ["CERAD", "BRAAK", "PLAQUES", "TANGLES", "ABETA_IHC", "TAU_IHC"]

In [6]:
path_to_models = path_to_configs + path_to_ext_val_results + "intersecting_gene_models/%s/"%dset
path_to_ext_val_predictions = path_to_configs + path_to_ext_val_results


# MTL

In [8]:
for i in range(100):
    print(i)

    fname = MTL_final_final_model
    path_to_model = path_to_models + "models/MTL/%s/%i/200.hdf5"%(MTL_final_final_model, i)

    MTL_model = keras.models.load_model(path_to_model, custom_objects={"ordloss_cur_params": ordloss(0), \
                "ignorenans_mse": ignorenans_mse, "cat_acc": ignorenans_categorical_accuracy(0), \
                "ignorenans_scaled_mse": ignorenans_scaled_mse})

    train_preds = MTL_model.predict(X_train)
    test_preds = MTL_model.predict(X_test)

    MTL_phenotype_output_mapping = {"BRAAK":0, "CERAD":1, "PLAQUES":2, "TANGLES":3, "ABETA_IHC":4, "TAU_IHC":5}
    
    path_to_train_preds = "%spredictions%s/%s/MTL/%s/train/"%(path_to_ext_val_predictions,dpath,dset, MTL_final_final_model)
    path_to_test_preds = "%spredictions%s/%s/MTL/%s/"%(path_to_ext_val_predictions,dpath,dset, MTL_final_final_model)
    
    for path in [path_to_train_preds, path_to_test_preds]:
        if not os.path.isdir(path):
            os.makedirs(path)
        
    train_pred_df = pd.DataFrame(np.hstack([train_preds[MTL_phenotype_output_mapping[phenotype]] for phenotype in phens]), columns=phens)
    train_pred_df.to_csv("%s/%i.csv"%(path_to_train_preds,i))

    test_pred_df = pd.DataFrame(np.hstack([test_preds[MTL_phenotype_output_mapping[phenotype]] for phenotype in phens]), columns=phens)
    test_pred_df.to_csv("%s/%i.csv"%(path_to_test_preds,i))
    
    K.clear_session()
    gc.collect()

0


/home/nbbwang/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1
2
3
4


OSError: Unable to open file (Unable to open file: name = '../../../md-ad_public_repo_data/external_validation/intersecting_gene_models/blood_gse63060/models/mtl/200_relu_[500, 100]_[50, 10]_0.100000_0.001000_0.001000_[1, 1]_0.100000_20/4/200.hdf5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)

# MLP

In [12]:


for i in range(10):
    print(i)
    #train_preds = []
    test_preds = []

    for p in phens:
        fname=baselines_final_final_model[p]
        path_to_model = path_to_models + "/models/MLP_baselines/%s/%s/%s/%i/200.hdf5"%(dset,p,fname, i)
        
        MLP_model = keras.models.load_model(path_to_model, custom_objects={"ordloss_cur_params": ordloss(0), \
                    "ignorenans_mse": ignorenans_mse, "cat_acc": ignorenans_categorical_accuracy(0), \
                    "ignorenans_scaled_mse": ignorenans_scaled_mse})

        #train_preds.append(MLP_model.predict(X_train))
        test_preds.append(MLP_model.predict(X_test))

        
        
        
    #path_to_train_preds = "%spredictions%s/%s/MLP_baselines/train/"%(path_to_ext_val_predictions,dpath,dset)
    path_to_test_preds = "%spredictions%s/%s/MLP_baselines/"%(path_to_ext_val_predictions,dpath,dset)
    
    for path in [path_to_test_preds]:
        if not os.path.isdir(path):
            os.makedirs(path)
        
    #train_pred_df = pd.DataFrame(np.hstack(train_preds), columns=phens)
    test_pred_df = pd.DataFrame(np.hstack(test_preds), columns=phens)

    #train_pred_df.to_csv("%s/%i.csv"%(path_to_train_preds,i))
    test_pred_df.to_csv("%s/%i.csv"%(path_to_test_preds,i))
    
    K.clear_session()
    gc.collect()

0


/home/nbbwang/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1


OSError: Unable to open file (Unable to open file: name = '../../../md-ad_public_repo_data/external_validation/intersecting_gene_models/blood_gse63060//models/mlp_baselines/blood_gse63060/cerad/200_relu_[500, 100]_[50, 10]_0.100000_0.000010_0.001000_[1, 1]_0.010000_20/1/200.hdf5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)